# Computational Tests

This notebook is intended for running different kinds of analyses that would validate our reconstruction.

[1. Subsystem Overview and Analysis](#subsystems) <br>
[2. Context-specific Model Generation](#context_specific) <br>
[3. Biomass prediction using exp. data](#biomass) <br>
[4. Flux Enrichment Analysis](#fea) <br>

## 1. Subsystem Overview and Analysis <a id='subsystems'></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from google_sheet import GoogleSheet

In [ ]:
##### ----- Generate Subsystems datasets from Google Sheet ----- #####

#Credential file
KEY_FILE_PATH = 'credentials.json'

#CHO Network Reconstruction + Recon3D_v3 Google Sheet ID
SPREADSHEET_ID = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_subsystems = 'Summary Systems'
sheet_reactions = 'Rxns'
subsystems = sheet.read_google_sheet(sheet_subsystems)
reactions_rec = sheet.read_google_sheet(sheet_reactions)

# Remove the total count
subsystems = subsystems.iloc[:-1, :]

In [ ]:
#### --- Pie Chart of the Amount of Reactions per System --- ####

subsystems["Number of Reactions"] = pd.to_numeric(subsystems["Number of Reactions"])

# Aggregate data by "System" and sum "Number of Reactions"
system_reactions = subsystems.groupby("System")["Number of Reactions"].sum()

# Set the figure and axes for more control
fig, ax = plt.subplots(figsize=(10, 7))

# Generate the pie chart again with the custom labels
patches, texts, autotexts = ax.pie(system_reactions, labels=system_reactions.index, autopct=lambda p: '{:.1f}%'.format(p) if p > 0 else '', startangle=140, colors=plt.cm.tab20.colors)

# Improve aesthetics
for text in texts + autotexts:
    text.set_color('black')
ax.set_title("Pie Chart of Number of Reactions by System", pad=30)  # Move the title upwards by increasing pad

# Set equal aspect ratio
ax.axis('equal')

# Save the plot
plt.savefig('../Plots/pie_chart_reactions_per_system.png')

# Display the plot
plt.show()

In [ ]:
# Generate the Sunburst Plot

fig = px.sunburst(subsystems, path=['System', 'Subsystems', 'Kegg Pathway'],
                  color='System')

# Convert to a Graph Objects figure
fig_go = go.Figure(fig)

# Update layout
fig_go.update_layout(width=1200, height=1000)

# Define font sizes
font_size_last_layer = 10
font_size_previous_layer = 15
default_font_size = 12

# Create a list to store font sizes
font_sizes = []

# Update font size for each level
for trace in fig_go.data:
    if isinstance(trace, go.Sunburst):
        for id in trace.ids:
            level = id.count("/")  # Determine level by the number of slashes in the id
            if level == 2:  # Last layer (Kegg Pathway)
                font_sizes.append(font_size_last_layer)
            elif level == 1:  # Previous layer (Subsystems)
                font_sizes.append(font_size_previous_layer)
            else:
                font_sizes.append(default_font_size)  # Default size for other layers

# Apply the font sizes to the figure
fig_go.update_traces(insidetextfont=dict(size=font_sizes))

# Save the figure
fig_go.write_html("../Plots/sunburst_subsystems.html")  # Save as interactive HTML file
fig_go.write_image("../Plots/sunburst_subsystems.png", width=1200, height=1000, scale=2)  # Increase resolution by setting scale parameter

# Show the plot
fig_go.show()

#### Auxotrophies

In [ ]:
from cobra.io import load_json_model

iCHO_path = "iCHO3595.json"
iCHO = load_json_model(iCHO_path)

In [ ]:
amino_acids = {
    "arginine": ["EX_arg_L_e", "EX_arg_D_e"],
    "asparagine": ["EX_asn_L_e", "EX_asn_D_e"],
    "cysteine": ["EX_cys_L_e", "EX_cys_D_e", "EX_Lcystin_e"],
    "histidine": ["EX_his_L_e", "EX_his_D_e"],
    "isoleucine": ["EX_ile_L_e", "EX_ile_D_e"],
    "leucine": ["EX_leu_L_e", "EX_leu_D_e"],
    "lysine": ["EX_lys_L_e", "EX_lys_D_e"],
    "methionine": ["EX_met_L_e", "EX_met_D_e"],
    "phenylalanine": ["EX_phe_L_e", "EX_phe_D_e"],
    "proline": ["EX_pro_L_e", "EX_pro_D_e"],
    "threonine": ["EX_thr_L_e", "EX_thr_D_e"],
    "tryptophan": ["EX_trp_L_e", "EX_trp_D_e"],
    "valine": ["EX_val_L_e", "EX_val_D_e"]
}
for amino_acid in amino_acids:
    # ----- Setup initial bounds -----
    for exchange_reaction in iCHO.exchanges:
        exchange_reaction.bounds = -10, 10

    # Arginine
    iCHO.reactions.get_by_id('GAUGE-R00557').bounds = 0, 10
    iCHO.reactions.get_by_id('GAUGE-R10107').bounds = 0, 0
    iCHO.reactions.get_by_id('GAUGE-R00558').bounds = 0, 0
    iCHO.reactions.get_by_id('GLYAMDTRc').bounds = 0, 0
    iCHO.reactions.get_by_id('GAUGE-R10107').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_valarggly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('ARGSL').bounds = 0, 0

    # Asparigine
    iCHO.reactions.get_by_id('ASNS1').bounds = 0, 0

    # Cysteine
    iCHO.reactions.get_by_id('r0129').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_cgly_e').bounds = 0, 0
    iCHO.reactions.get_by_id('AMPTASECG').bounds = -10, 0
    iCHO.reactions.get_by_id('AMPTASECGe').bounds = -10, 0
    iCHO.reactions.get_by_id('CYSTGL').bounds = -10, 0
    iCHO.reactions.get_by_id('EX_HC00250_e').bounds = 0, 0
    iCHO.reactions.get_by_id('EX_sfcys_e').bounds = 0, 0

    # Histidine
    iCHO.reactions.get_by_id('VALTRPVALr').bounds = 0,0 

    # Isoleucine - DONE
    iCHO.reactions.get_by_id('EX_CE2916_e').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_CE2915_e').bounds = 0,0 
    iCHO.reactions.get_by_id('ILETA').bounds = 0, 10
    iCHO.reactions.get_by_id('ILETAm').bounds = 0, 10

    # Leucine
    iCHO.reactions.get_by_id('LEULEULAPc').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_leugly_e').bounds = 0,0 
    iCHO.reactions.get_by_id('EX_glyleu_e').bounds = 0,0 
    iCHO.reactions.get_by_id('LEUTA').bounds = 0, 10 
    iCHO.reactions.get_by_id('LEUTAm').bounds = 0, 10 
    iCHO.reactions.get_by_id('EX_CE5797_e').bounds = 0, 0

    # Lysine
    iCHO.reactions.get_by_id('EX_biocyt_e').bounds = 0,0 

    # Methionine
    iCHO.reactions.get_by_id('METS').bounds = -10, 0 
    iCHO.reactions.get_by_id('BHMT').bounds = -10, 0 
    iCHO.reactions.get_by_id('GAUGE-R00648').bounds = 0, 10 
    iCHO.reactions.get_by_id('UNK2').bounds = -10, 0 
    iCHO.reactions.get_by_id('UNK3').bounds = -10, 0 
    iCHO.reactions.get_by_id('TYRA').bounds = -10, 0 
    #iCHO.reactions.get_by_id('GAUGE-R06895').bounds = 0, 0 # Curated by MR with 1 score / Erased from the reconstruction

    # Phenylalanine
    iCHO.reactions.get_by_id('EX_CE5786_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_pheleu_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_glyphe_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE2917_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE5786_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_CE5789_e').bounds = 0, 0 
    iCHO.reactions.get_by_id('EX_phpyr_e').bounds = 0, 0 

    # Proline
    iCHO.reactions.get_by_id('EX_glypro_e').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_progly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('P5CR').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRxm').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRx').bounds = 0, 0
    iCHO.reactions.get_by_id('P5CRm').bounds = 0, 0
    iCHO.reactions.get_by_id('r1453').bounds = 0, 10

    # Threonine
    #iCHO.reactions.get_by_id('THRS').bounds = 0,0  # Erased from the reconstruction


    # Tryptophan
    iCHO.reactions.get_by_id('NBAHH_ir').bounds = 0,0  # Histidine hydrolase

    # Valine
    iCHO.reactions.get_by_id('EX_valarggly_e').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_vallystyr_e').bounds = 0, 10
    iCHO.reactions.get_by_id('VALTA').bounds = 0, 10
    iCHO.reactions.get_by_id('VALTAm').bounds = 0, 10
    iCHO.reactions.get_by_id('EX_valval_e').bounds = 0, 10
    # -----------------------------------------

    for exchange_reaction in iCHO.exchanges:
        if exchange_reaction.id in amino_acids[amino_acid]:
            exchange_reaction.bounds = 0, 10
            # print(amino_acid, exchange_reaction.id, amino_acids[amino_acid])
    sol = iCHO.optimize()
    print(amino_acid, sol.objective_value)


In [ ]:
iCHO.metabolites.leu_L_c.summary()

#### Gene Deletion test

In [ ]:
import pandas as pd
from cobra.io import load_json_model, read_sbml_model
from cobra.flux_analysis import single_gene_deletion

#iCHO = load_json_model('iCHO3595_unblocked.json')
#iCHO1766 = read_sbml_model('../Data/Reconciliation/models/iCHOv1_final.xml')
iCHO2291 = read_sbml_model('../Data/Reconciliation/models/iCHO2291.xml')
#iCHO2441 = read_sbml_model('../Data/Reconciliation/models/iCHO2441.xml')

In [ ]:
gene_deletion_results = single_gene_deletion(iCHO2291)
essentiality_threshold = 0.01
essential_genes = gene_deletion_results[gene_deletion_results['growth'] < essentiality_threshold]

In [ ]:
# Print the essential genes
print(f"Number of essential genes for biomass: {len(essential_genes)}")
print("Essential genes for biomass production:")
for gene_idx in essential_genes.index:
    gene = iCHO2291.genes[gene_idx]
    print(f"{gene.id}: {gene.name}")

In [ ]:
from cobra.flux_analysis.variability import find_essential_genes

In [ ]:
sim_essential_genes = find_essential_genes(model)

In [ ]:
# Load the dataset for the experimentally validated essential genes

exp_essential_genes = pd.read_csv('../Data/Gene_Essentiality/cho_essential_genes.csv', sep='\t')
exp_essential_genes['gene'] = exp_essential_genes['gene'].str.replace('__1$', '', regex=True)

In [ ]:
exp_essential_genes

In [ ]:
# Extract all gene names
all_gene_names = set()
for names in exp_essential_genes['gene']:
    for name in names.split('|'): #Extract the names of the genes with two aliases
        all_gene_names.add(name)

In [ ]:
essential_genes_model = []
for g in model.genes:
    if g.name in all_gene_names:
        essential_genes_model.append(g)

In [ ]:
set1 = set(sim_essential_genes)
set2 = set(essential_genes_model)

# Find shared elements
shared_elements = set1.intersection(set2)

# Find unique elements
unique_in_list1 = set1.difference(set2)
unique_in_list2 = set2.difference(set1)

In [ ]:
print("Shared genes between Exp Validated Essential Genes and Simulated Essential Genes:", len(shared_elements))
print("Unique genes in Simulated Essential Genes:", len(unique_in_list1))
print("Unique genes in Exp Validated Essentail Genes:", len(unique_in_list2))

##### Essential for growth genes and comparison to (An optimised genome-wide, virus free CRISPR screen for mammalian cells)
https://pubmed.ncbi.nlm.nih.gov/34935002/

## 2. Context-specific Model Generation <a id='context_specific'></a>

Here we use a matrix generated with rmf_CADRE to generate each context_specific model for each one of the conditions.

In [ ]:
import pickle
import pandas as pd
from scipy.io import loadmat
from time import process_time

import pymCADRE
from pymCADRE.rank import rank_reactions
from pymCADRE.check import check_model_function
from pymCADRE.prune import prune_model

import cobra
from cobra.io import load_json_model

### Define Intervals for Context-specific model generation
Here we define the set of constraints in the uptake and secretion rates that we want to incorporate to our context specific models

#### Model Loading and Constraining

In [ ]:
# Load generic model
model = load_json_model('iCHO3595_unblocked.json')

In [ ]:
# Load uptake and secretion rate "Intervals dict

with open('../Data/Uptake_Secretion_Rates/uptake_secretion_intrvl_wt_dict.pkl', 'rb') as file:
    uptsec_intrvl_wt = pickle.load(file)

with open('../Data/Uptake_Secretion_Rates/uptake_secretion_intrvl_zela_dict.pkl', 'rb') as file:
    uptsec_intrvl_zela = pickle.load(file)

In [ ]:
temp_dict = uptsec_intrvl_zela
time = 'P2 to P4'

with model as model:
    model.reactions.biomass_cho_s.bounds = temp_dict['exp_growth_rate'][time]
    for rxn in model.boundary:
        
        # Models that are forced to secrete ethanol are not feasible
        if rxn.id == 'EX_etoh_e':
            rxn.bounds = (-0.1,0.1)
            continue

        # Replace the lower and upper bound with experimental data
        if rxn.id in temp_dict.keys():
            rxn.bounds = temp_dict[rxn.id][time]
            continue
        
        # Keep boundaries open for essential metabolites
        if rxn.id == 'EX_h2o_e':
            rxn.bounds = (-1000,1000)
            continue
        if rxn.id == 'EX_h_e':
            rxn.bounds = (-1000,1000)
            continue
        if rxn.id == 'EX_o2_e':
            rxn.bounds = (-1000,1000)
            continue
        if rxn.id == 'EX_hco3_e':
            rxn.bounds = (-1000,1000)
            continue
        if rxn.id == 'EX_so4_e':
            rxn.bounds = (-1000,1000)
            continue
        if rxn.id == 'EX_pi_e':
            rxn.bounds = (-1000,1000)
            continue

        # Boundaries from Sink reactions on iCHO_v1 (100 times lower)
        if rxn.id == 'SK_Asn_X_Ser_Thr_r':
            rxn.bounds = (-0.001,1000)
            continue
        if rxn.id == 'SK_Tyr_ggn_c':
            rxn.bounds = (-0.001,1000)
            continue
        if rxn.id == 'SK_Ser_Thr_g':
            rxn.bounds = (-0.001,1000)
            continue
        if rxn.id == 'SK_pre_prot_r':
            rxn.bounds = (-0.0006,1000)
            continue

        # Close uptake rates for the rest of the boundaries
        if rxn.id.startswith("EX_"):
            rxn.bounds = (0,1000) 
        if rxn.id.startswith("SK_"):
            rxn.bounds = (0,1000)
        if rxn.id.startswith("DM_"):
            rxn.bounds = (0,1000)
        
    pfba_solution = cobra.flux_analysis.pfba(model)
    pfba_obj = pfba_solution.fluxes[objective]
    print(pfba_obj)

### Loading Ubiquity and Confidence Scores

In [ ]:
# Load the .mat file
mat_data = loadmat('../Data/Context_specific_models/ubiData.mat')

# Access the ubiData structure
ubiData = mat_data['ubiData']

# Accessing individual fields within ubiData
ubiScores = ubiData['ubiScores'][0, 0]
uScore = ubiData['uScore'][0, 0]
rxns = ubiData['rxns'][0, 0]
Condition = ubiData['Condition'][0, 0]

# Print or process the data as needed
print(ubiScores.shape)
print(uScore.shape)
print(rxns.shape)
print(Condition.shape)

In [ ]:
for i,cond in enumerate(Condition[0]):
    if "WT_P4" in cond[0]:
        print(cond[0], i)
    elif "ZeLa_P4" in cond[0]:
        print(cond[0], i)

In [ ]:
Condition[0][0]

In [ ]:
# Ubiquity score list
U = list(ubiScores[:,0])

In [ ]:
# Conf Score List
conf_scores = pd.read_csv('../Data/Context_specific_models/confidence_scores.csv', header=None).iloc[:, 0].tolist()

In [ ]:
# Protected Reations
high_confidence_reactions = ['biomass_cho_s', 'DNAsyn', 'LipidSyn_cho_s', 'PROTsyn_cho_s', 'RNAsyn_cho_s', 'EX_bhb_e', 'EX_nh4_e', 'EX_ac_e', 'EX_ala_L_e',
    'EX_arg_L_e', 'EX_asn_L_e', 'EX_asp_L_e', 'EX_2hb_e', 'EX_cit_e', 'EX_cys_L_e', 'EX_etoh_e', 'EX_for_e', 'EX_fum_e', 'EX_glc_e', 'EX_glu_L_e',
    'EX_gln_L_e', 'EX_gly_e', 'EX_his_L_e', 'EX_4hpro_e', 'EX_ile_L_e', 'EX_lac_L_e', 'EX_leu_L_e', 'EX_lys_L_e', 'EX_mal_L_e', 'EX_met_L_e',
    'EX_phe_L_e', 'EX_pro_L_e', 'EX_5oxpro_e', 'EX_pyr_e', 'EX_ser_L_e', 'EX_thr_L_e', 'EX_trp_L_e', 'EX_tyr_L_e', 'EX_val_L_e', 'EX_h2o_e',
    'EX_h_e', 'EX_o2_e', 'EX_hco3_e', 'EX_so4_e', 'EX_pi_e', 'SK_Asn_X_Ser_Thr_r', 'SK_Tyr_ggn_c', 'SK_Ser_Thr_g', 'SK_pre_prot_r'
]

### Reduced model generation

In [ ]:
##############################################
# Rank reactions
##############################################

print('Processing inputs and ranking reactions...')
GM, C, NC, P, Z, model_C = rank_reactions(model, U, conf_scores, high_confidence_reactions, method=1)

In [ ]:
# Output the results
print("Generic Model:", GM)
print("Core Reactions:", len(C))
print("Non-Core Reactions:", len(NC))
print("Ranked Non-Core Reactions:", len(P))
print("Zero-Expression Reactions:", len(Z))
print("Core Reactions in Original Model:", len(model_C))

In [ ]:
from cobra.util import create_stoichiometric_matrix

def retrieve_precursor_metabolites(model, protected_rxns):
    """
    Retrieve precursor metabolites from a specified list of reactions in a metabolic model.

    Args:
        model: cobra.Model - The metabolic model.
        protected_rxns: list - A list of reaction IDs for which precursor metabolites need to be retrieved.

    Returns:
        precursor_mets: list - A list of precursor metabolites.
    """

    # Create a list of all the reactions in the model
    model_reactions = [r.id for r in model.reactions]
    
    # Ensure that the protected reactions list is not empty
    if protected_rxns:
        
        #Define a list to store the indices of the protected reactions
        rxn_indices = []
        
        for rxn in protected_rxns:
            # Ensure that the protected reaction is included in the model
            if rxn in model_reactions:
                # Get the indices of the protected reactions in the model, only if the reaction exists
                rxn_indices.append(model.reactions.index(model.reactions.get_by_id(rxn)))

        # Create stoichiometric matrix S
        S = create_stoichiometric_matrix(model)
        
        # Retrieve precursor metabolites: metabolites with a negative coefficient in the stoichiometric matrix
        precursor_mets = [
            model.metabolites[i] 
            for i in range(len(model.metabolites)) 
            if any(S[i, idx] < 0 for idx in rxn_indices)
        ]

        return precursor_mets
    else:
        return []

In [ ]:
precursor_mets = retrieve_precursor_metabolites(model, high_confidence_reactions)

In [ ]:
##################################################
# Define inputs to the model pruning step
##################################################

# Define core vs. non-core ratio threshold for removing reactions
eta = 1 / 3

# Check functionality of generic model
genericStatus = check_model_function(GM, 'required_mets', precursor_mets)[0]
print(genericStatus)

In [ ]:
# Constraint the model with exp data

time = 'P2 to P4'

GM.solver = 'gurobi'
objective = 'biomass_cho_s' # 'biomass_cho'
GM.objective = objective

GM.reactions.biomass_cho_s.bounds = temp_dict['exp_growth_rate'][time]
for rxn in GM.boundary:
    
    # Models that are forced to secrete ethanol are not feasible
    if rxn.id == 'EX_etoh_e':
        rxn.bounds = (-0.1,0.1)
        continue

    # Replace the lower and upper bound with experimental data
    if rxn.id in temp_dict.keys():
        rxn.bounds = temp_dict[rxn.id][time]
        continue
    
    # Keep boundaries open for essential metabolites
    if rxn.id == 'EX_h2o_e':
        rxn.bounds = (-1000,1000)
        continue
    if rxn.id == 'EX_h_e':
        rxn.bounds = (-1000,1000)
        continue
    if rxn.id == 'EX_o2_e':
        rxn.bounds = (-1000,1000)
        continue
    if rxn.id == 'EX_hco3_e':
        rxn.bounds = (-1000,1000)
        continue
    if rxn.id == 'EX_so4_e':
        rxn.bounds = (-1000,1000)
        continue
    if rxn.id == 'EX_pi_e':
        rxn.bounds = (-1000,1000)
        continue

    # Boundaries from Sink reactions on iCHO_v1 (100 times lower)
    if rxn.id == 'SK_Asn_X_Ser_Thr_r':
        rxn.bounds = (-0.001,1000)
        continue
    if rxn.id == 'SK_Tyr_ggn_c':
        rxn.bounds = (-0.001,1000)
        continue
    if rxn.id == 'SK_Ser_Thr_g':
        rxn.bounds = (-0.001,1000)
        continue
    if rxn.id == 'SK_pre_prot_r':
        rxn.bounds = (-0.0006,1000)
        continue

    # Close uptake rates for the rest of the boundaries
    if rxn.id.startswith("EX_"):
        rxn.bounds = (0,1000) 
    if rxn.id.startswith("SK_"):
        rxn.bounds = (0,1000)
    if rxn.id.startswith("DM_"):
        rxn.bounds = (0,1000)
    
pfba_solution = cobra.flux_analysis.pfba(GM)
pfba_obj = pfba_solution.fluxes[objective]
print(pfba_obj)

In [ ]:
if genericStatus:
    print('Generic model passed precursor metabolites test')

    ##############################################################
    # If generic functionality test is passed, prune reactions
    ###############################################################
    print('Pruning reactions...')
    t0 = process_time()
    PM, cRes = prune_model(GM, P, C, Z, eta, precursor_mets, salvage_check=0, method=1)
    # Stop the stopwatch / counter
    t_stop = process_time()
    # compute elapsed time
    pruneTime = t_stop - t0

else:
    print('Generic model failed precursor metabolites test!!')

### ATP Generation loops removal

In [1]:
from itertools import chain
import pickle
import numpy as np
from optlang.symbolics import Zero

import cobra
from cobra.util import create_stoichiometric_matrix
from cobra.io import load_json_model

#### Loading and Constraining Model

In [2]:
model = load_json_model('iCHO3595_unblocked.json')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04


In [3]:
with open('../Data/Uptake_Secretion_Rates/uptake_secretion_raw_zela_dict.pkl', 'rb') as file:
    uptsec_zela = pickle.load(file)

for rxn in model.boundary:
    if rxn.id in ['EX_h2o_e', 'EX_h_e', 'EX_o2_e', 'EX_hco3_e', 'EX_so4_e', 'EX_pi_e']:
        rxn.bounds = (-1000, 1000)
    elif rxn.id in ['SK_Asn_X_Ser_Thr_r', 'SK_Tyr_ggn_c', 'SK_Ser_Thr_g', 'SK_pre_prot_r']:
        rxn.bounds = (-0.001, 1000)
    elif rxn.id.startswith(("EX_", "SK_", "DM_")):
        rxn.bounds = (0, 1000)  # Close uptake rates for others

for reaction in model.reactions:
    if reaction.id == 'EX_etoh_e': #Model creates infeasible solutions when secreting etoh
        continue
    elif reaction.id == 'ATPM': # Add ATP Maintenance Cost
        reaction.lower_bound = 0
    for r,v in uptsec_zela.items():
        if reaction.id == r:
            reaction.upper_bound = 1000
            reaction.lower_bound = v[('U6','P2 to P4')]

#### ATP loops removal

In [ ]:
#########################################
# Thanasis' original version
#########################################


# Get the maximum flux through the oxidatitve phosphoryaltion pathway. Get the maximum flux of the objectives, biomass and IgG. 
# If a reaction that consumes ATP (negative stoichiometric coefficient), has a flux value below a cutoff in both maximisation of biomass
# IgG and ox. pho. pathway, then the reaction doesn't provide any useable ATP (cyclic)

step6_safe_rxn_rmv = []

protected_subsystems = [ # Why protecting these subsystems
    'BIOMASS', 'Product Formation', 'OXIDATIVE PHOSPHORYLATION', 'CITRIC ACID CYCLE', 'GLYCOLYSIS/GLUCONEOGENESIS',
    'PYRUVATE METABOLISM', 'PENTOSE PHOSPHATE PATHWAY', 'GLUTAMATE METABOLISM', 'NAD METABOLISM',
    'TRANSPORT, MITOCHONDRIAL', 'TRANSPORT, ENDOPLASMIC RETICULAR', 'ARGININE AND PROLINE METABOLISM',
    'UBIQUINONE SYNTHESIS', 'VALINE, LEUCINE, AND ISOLEUCINE METABOLISM', 'TRYPTOPHAN METABOLISM',
    'METHIONINE AND CYSTEINE METABOLISM', 'TYROSINE METABOLISM', 'GLYCINE, SERINE, AND THREONINE METABOLISM',
    'HISTIDINE METABOLISM', 'ALANINE AND ASPARTATE METABOLISM', 'PHENYLALANINE METABOLISM',
    'LYSINE METABOLISM', 'TYROSINE METABOLISM', 'N-GLYCAN BIOSYNTHESIS', 'PROTEIN PRODUCTION', 'SECRETORY PATHWAY'
]


# biomass and igg reactions of the model; we want to ensure their production after removals of any ATP cycles
biomass_rxn = 'biomass_cho_s'
igg_rxn = 'igg_formation'

model.objective = biomass_rxn
biomass_control = model.optimize('maximise')
model.objective = igg_rxn
igg_control = model.optimize('maximise')
print("Original: Biomass: ", round(biomass_control.objective_value, 5), "IgG: ", round(igg_control.objective_value, 5))

model_temp = model.copy()
atp_related_mets = ['atp_c', 'atp_n', 'atp_m', 'atp_x'] # What about atp_l, atp_g, atp_r, atp_e
atp_met_indices = [model_temp.metabolites.get_by_id(met) for met in atp_related_mets]

# Find the reactions associated with these metabolites
atp_rxns_indices = []
for met in atp_met_indices:
    for reac in met.reactions:
        atp_rxns_indices.append(reac.id)
atp_rxns_indices = list(set(atp_rxns_indices))

# Determine which reactions are consuming ATP (negative stoichiometry for ATP)
atp_consumption_idx = []
stoich_matrix = create_stoichiometric_matrix(model_temp)
for atp_rxn in atp_rxns_indices:
    reaction = model_temp.reactions.get_by_id(atp_rxn)
    if not reaction.reversibility:  # Only irreversible reactions
        for met in atp_met_indices:
            if stoich_matrix[model_temp.metabolites.index(met), model_temp.reactions.index(reaction)] < 0:
                atp_consumption_idx.append(atp_rxn)
                break

# Perform FBA to get maximum ATP production
model_temp.objective = 0
oxpho_rxns = [r for r in model_temp.reactions if 'OXIDATIVE PHOSPHORYLATION' in r.subsystem]
reaction_variables = ((rxn.forward_variable, rxn.reverse_variable) for rxn in oxpho_rxns)
variables = chain(*reaction_variables)
model_temp.objective = model_temp.problem.Objective( Zero, direction="max", sloppy=True, name="maxATP" )
model_temp.objective.set_linear_coefficients({v: 1.0 for v in variables})
print("objective function", model_temp.objective.expression)
solution_maxATP = model_temp.optimize()

for obj in [biomass_rxn, igg_rxn]:
    model_temp.objective = obj
    sol = model_temp.optimize('maximize')
    atp_rate = np.vstack([sol[atp_consumption_idx], solution_maxATP[atp_consumption_idx]])

    # Find ATP consuming reactions with low rates
    atp_rxn_lowFlux = [id for id, rate in zip(atp_consumption_idx, np.sum(atp_rate, axis=0)) if rate <= 2e-12]

    atp_loop_rxn_rmv = []
    for rxn in atp_rxn_lowFlux:
        if model_temp.reactions.get_by_id(rxn).subsystem not in protected_subsystems:
            atp_loop_rxn_rmv.append(rxn)

    # Filter out exchange reactions
    exchange_reactions = [r.id for r in model_temp.exchanges]
    atp_loop_rxn_rmv = [r_id for r_id in atp_loop_rxn_rmv if r_id not in exchange_reactions]

    # Ensure ATP maintenance reaction is not deleted
    id_atp_maint = model_temp.reactions.get_by_id('ATPM').id
    atp_loop_rxn_rmv = [r_id for r_id in atp_loop_rxn_rmv if r_id != id_atp_maint]

    if obj == biomass_rxn:
        atp_loop_rxns_bio = atp_loop_rxn_rmv
    elif obj == igg_rxn:
        atp_loop_rxns_igg = atp_loop_rxn_rmv

# Print the reactions to be deleted
atp_loop_rxns = list(set(atp_loop_rxns_igg).intersection(atp_loop_rxns_bio))
print("Reactions to be deleted:", len(atp_loop_rxns))

# Remove reactions from the model and check that Biomass and IgG are produced
model_temp.objective = biomass_rxn
sol_bio_control = model_temp.optimize('maximize')
model_temp.objective = igg_rxn
sol_igg_control = model_temp.optimize('maximize')
model_temp_temp = model_temp.copy()

for rxn_to_rmv in atp_loop_rxns:
    model_temp_temp.remove_reactions(rxn_to_rmv)
    model_temp_temp.objective = biomass_rxn
    sol_bio = model_temp_temp.optimize('maximize')
    model_temp_temp.objective = igg_rxn
    sol_igg = model_temp_temp.optimize('maximize')
    if round(sol_bio.objective_value, 5) == round(sol_bio_control.objective_value, 5) and round(sol_igg.objective_value, 5) == round(sol_igg_control.objective_value, 5):
        step6_safe_rxn_rmv.append(rxn_to_rmv)

model_temp.remove_reactions(step6_safe_rxn_rmv)
print("Reactions consuming ATP in a cyclic fashion AND are safe to remove: ", len(step6_safe_rxn_rmv))
print("Model size after the removal of ATP cyclic reactions: Reactions: ", len(model_temp.reactions), "Metabolites: ", len(model_temp.metabolites))
model_temp.objective = biomass_rxn
sol_bio = model_temp.optimize('maximize')
model_temp.objective = igg_rxn
sol_igg = model_temp.optimize('maximize')
print("Old Bio: ", round(sol_bio_control.objective_value, 3), "New Bio: ", round(sol_bio.objective_value, 3), "Old IgG: ", round(sol_igg_control.objective_value, 5), "New IgG: ", round(sol_igg.objective_value, 5))
model = model_temp.copy()

In [4]:
#########################################
# Pablo's modification
#########################################
from tqdm import tqdm


def get_atp_related_reactions(model, atp_mets):
    """ Get reactions associated with ATP metabolites. """
    atp_rxns_indices = []
    for met in atp_mets:
        for reac in met.reactions:
            atp_rxns_indices.append(reac.id)
    return list(set(atp_rxns_indices))

def find_low_flux_reactions(model, atp_rxns_indices, atp_met_indices, stoich_matrix, consumption_cutoff=2e-12):
    """ Identify reactions with low ATP consumption flux. """
    atp_consumption_idx = []
    for atp_rxn in atp_rxns_indices:
        reaction = model.reactions.get_by_id(atp_rxn)
        for met in atp_met_indices:
            if stoich_matrix[model.metabolites.index(met), model.reactions.index(reaction)] < 0:
                atp_consumption_idx.append(atp_rxn)
                break
    return atp_consumption_idx

def perform_fba_for_max_atp(model, oxpho_rxns):
    """ Set FBA objective to maximize ATP production. """
    reaction_variables = ((rxn.forward_variable, rxn.reverse_variable) for rxn in oxpho_rxns)
    variables = chain(*reaction_variables)
    model.objective = model.problem.Objective(Zero, direction="max", sloppy=True, name="maxATP")
    model.objective.set_linear_coefficients({v: 1.0 for v in variables})
    return model.optimize()

def remove_cyclic_atp_reactions(model, atp_rxns_to_remove, biomass_rxn):
    """ Remove reactions consuming ATP in a cyclic fashion and check for safety. """
    
    safe_rxn_rmv = []
    safe_rxn_irrev = []

    # Copy the original model
    temp_model = model.copy()

    # Set the objective for the temporary model
    temp_model.objective = biomass_rxn
    sol_bio_control = temp_model.optimize()

    # Cumulative list of safe reactions to remove or modify
    candidate_safe_rmv = []
    candidate_safe_irrev = []

    for rxn_id in tqdm(atp_rxns_to_remove, desc="Processing reactions"):
        with temp_model:
            print(f"\nTesting reaction: {rxn_id}")
            
            # Temporarily knock out the reaction
            temp_model.reactions.get_by_id(rxn_id).knock_out()
            sol_bio_temp = temp_model.optimize()

            if (round(sol_bio_temp.objective_value, 5) == round(sol_bio_control.objective_value, 5)):
                print(f"Reaction {rxn_id} knockout is safe.")
                
                # Temporarily add it to the candidate list
                candidate_safe_rmv.append(rxn_id)

                print(f"Added {rxn_id} to the candidate removal list.")
                
                # Test cumulative effect of all candidate removals
                temp_model.remove_reactions(candidate_safe_rmv)
                sol_bio_cumulative = temp_model.optimize()
                
                # If the cumulative effect is still safe, add to the final removal list
                if round(sol_bio_cumulative.objective_value, 5) == round(sol_bio_control.objective_value, 5):
                    safe_rxn_rmv = list(candidate_safe_rmv)
                    print(f"Cumulative effect of {rxn_id} is safe. Updated safe removal list.")
                else:
                    # If not safe, remove the last added reaction from candidates
                    candidate_safe_rmv.remove(rxn_id)
                    print(f"Removal of {rxn_id} is not safe, reverting.")
            
            else:
                print(f"Knockout of {rxn_id} affects biomass production. Testing for irreversibility.")
                
                # Try to make the reaction irreversible if removal is not safe
                if temp_model.reactions.get_by_id(rxn_id).lower_bound != 0:
                    temp_model.reactions.get_by_id(rxn_id).bounds = (0, 1000)
                    sol_bio_temp = temp_model.optimize()
                    
                    if (round(sol_bio_temp.objective_value, 5) == round(sol_bio_control.objective_value, 5)):
                        print(f"Making reaction {rxn_id} irreversible is safe.")
                        
                        candidate_safe_irrev.append(rxn_id)
                        
                        # Test cumulative effect of all candidate modifications
                        for rxn in candidate_safe_irrev:
                            temp_model.reactions.get_by_id(rxn).bounds = (0, 1000)
                        sol_bio_cumulative = temp_model.optimize()

                        # If the cumulative effect is still safe, add to the final modification list
                        if round(sol_bio_cumulative.objective_value, 5) == round(sol_bio_control.objective_value, 5):
                            safe_rxn_irrev = list(candidate_safe_irrev)
                            print(f"Safe to make {rxn_id} irreversible. Updated irreversible list.")
                        else:
                            # If not safe, undo the last added irreversible modification
                            candidate_safe_irrev.remove(rxn_id)
                            print(f"Making {rxn_id} irreversible is not safe, reverting.")

    # Remove reactions that are safe to remove from the original model
    model.remove_reactions(safe_rxn_rmv)

    # Make irreversible reactions that can be safely modified
    for rxn in safe_rxn_irrev:
        if model.reactions.get_by_id(rxn).lower_bound != 0:
            model.reactions.get_by_id(rxn).bounds = (0, 1000)
    
    return model, safe_rxn_rmv, safe_rxn_irrev

# Set objectives
biomass_rxn = 'biomass_cho_s'
model.objective = biomass_rxn
biomass_control = model.optimize('maximize')

# Find ATP related reactions
atp_related_mets = ['atp_c', 'atp_n', 'atp_m', 'atp_x', 'atp_l', 'atp_g', 'atp_r', 'atp_e']
atp_met_indices = [model.metabolites.get_by_id(met) for met in atp_related_mets]
atp_rxns_indices = get_atp_related_reactions(model, atp_met_indices)

# Create stoichiometric matrix
stoich_matrix = create_stoichiometric_matrix(model)
atp_consumption_idx = find_low_flux_reactions(model, atp_rxns_indices, atp_met_indices, stoich_matrix)

# Maximize ATP production
oxpho_rxns = [r for r in model.reactions if 'OXIDATIVE PHOSPHORYLATION' in r.subsystem]
solution_maxATP = perform_fba_for_max_atp(model, oxpho_rxns)

# Check flux of ATP consuming reactions
atp_loop_rxn_rmv = []

model.objective = biomass_rxn
sol = model.optimize('maximize')
atp_rate = np.vstack([sol[atp_consumption_idx], solution_maxATP[atp_consumption_idx]])

# Find low flux ATP reactions
atp_rxn_lowFlux = [rxn_id for rxn_id, rate in zip(atp_consumption_idx, np.sum(atp_rate, axis=0)) if rate <= 2e-12]

# Filter protected subsystems and exchanges
protected_subsystems = ['BIOMASS', 'OXIDATIVE PHOSPHORYLATION']
atp_loop_rxn_rmv += [rxn for rxn in atp_rxn_lowFlux if model.reactions.get_by_id(rxn).subsystem not in protected_subsystems]

# Ensure ATPM and exchange reactions are not removed
atp_loop_rxn_rmv = [r_id for r_id in atp_loop_rxn_rmv if r_id != 'ATPM' and not r_id.startswith("EX_")]

# Remove duplicates in the list
atp_loop_rxn_rmv = set(atp_loop_rxn_rmv)

# Remove cyclic ATP reactions and check safety
pruned_model, safe_rxn_rmv, safe_rxn_irrev = remove_cyclic_atp_reactions(model, atp_loop_rxn_rmv, biomass_rxn)

print(f"Reactions consuming ATP in a cyclic fashion and safe to remove: {len(safe_rxn_rmv)}")
print(f"Reactions consuming ATP in a cyclic fashion and safe to make irreversible: {len(safe_rxn_irrev)}")

# Final biomass
pruned_model.objective = biomass_rxn
sol_bio = pruned_model.optimize('maximize')

print(f"Old Bio: {biomass_control.objective_value:.3f}, New Bio: {sol_bio.objective_value:.3f}")

Read LP format model from file /var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/tmpcl6976q9.lp
Reading time = 0.05 seconds
: 4764 rows, 17114 columns, 72396 nonzeros


Processing reactions:   0%|                     | 1/759 [00:00<02:28,  5.09it/s]


Testing reaction: GDCA3Sabc
Reaction GDCA3Sabc knockout is safe.
Added GDCA3Sabc to the candidate removal list.
Cumulative effect of GDCA3Sabc is safe. Updated safe removal list.

Testing reaction: r1344
Reaction r1344 knockout is safe.
Added r1344 to the candidate removal list.


Processing reactions:   0%|                     | 2/759 [00:01<07:07,  1.77it/s]

Cumulative effect of r1344 is safe. Updated safe removal list.

Testing reaction: PROSTGE1t3
Reaction PROSTGE1t3 knockout is safe.
Added PROSTGE1t3 to the candidate removal list.


...


Testing reaction: RTOTAL3FATPc
Reaction RTOTAL3FATPc knockout is safe.
Added RTOTAL3FATPc to the candidate removal list.
Cumulative effect of RTOTAL3FATPc is safe. Updated safe removal list.


Processing reactions: 100%|██████████████████▉| 758/759 [47:32<00:07,  7.02s/it]


Testing reaction: HMR_0267
Reaction HMR_0267 knockout is safe.
Added HMR_0267 to the candidate removal list.
Cumulative effect of HMR_0267 is safe. Updated safe removal list.


Processing reactions: 100%|███████████████████| 759/759 [47:40<00:00,  3.77s/it]


NameError: name 'safe_rxns_to_remove' is not defined

In [5]:
print(f"Reactions consuming ATP in a cyclic fashion and safe to remove: {len(safe_rxn_rmv)}")
print(f"Reactions consuming ATP in a cyclic fashion and safe to make irreversible: {len(safe_rxn_irrev)}")

# Final biomass
pruned_model.objective = biomass_rxn
sol_bio = pruned_model.optimize('maximize')

print(f"Old Bio: {biomass_control.objective_value:.3f}, New Bio: {sol_bio.objective_value:.3f}")

Reactions consuming ATP in a cyclic fashion and safe to remove: 758
Reactions consuming ATP in a cyclic fashion and safe to make irreversible: 0
Old Bio: 0.047, New Bio: 0.047


In [9]:
safe_rxn_rmv

['GDCA3Sabc',
 'r1344',
 'PROSTGE1t3',
 'PTPATe',
 'FACOAL240',
 'r0321',
 'HMR_0156',
 'ACCOALm',
 'HMR_2989',
 'PI4P5Kn',
 'FACOAL161x',
 'PEFLIPm',
 'FACOAL161',
 'HMR_0211',
 'r1356',
 'HMR_2942',
 'PRPPS',
 'r1103',
 'ARACHFATPc',
 'r1530',
 'CE2211te',
 'HMR_0419',
 'ICDCHOLabc',
 'PIK5',
 'NDPK9n',
 'PSFLIPm',
 'FAOXC16DCr',
 'PYDAMK',
 'CERK',
 'ISOCHOLabc',
 'HMR_0209',
 'r1346',
 'ATAx',
 '4PYRDX',
 'HEX2',
 'TMDK1m',
 '11DOCRTSLte',
 'AHANDROSTANtr',
 'KHK3',
 'FACOAL181x',
 'RE1441G',
 'NDPK7m',
 'PTPAT',
 'ADNK1n',
 'FACOAL2251',
 'HDCA24Gte',
 'r0812',
 'r1097',
 'HMR_7161',
 'HMR_2973',
 'HMR_2968',
 'r1095',
 'FACOAL1831x',
 '3DHDCAabc',
 'HMR_0323',
 'GLYK',
 'HMR_0311',
 'HMR_0343',
 'r0679',
 'HMR_0391',
 'CHSTEROLt',
 'r1359',
 'SPHS1Pt2e',
 'CE7096te',
 'HMR_0395',
 'NDPK10',
 'r1116',
 'GLYCK2',
 'NDPK2m',
 'r0680',
 'RE2635C',
 'r0456',
 'BiGGRxn56',
 'FPGSm',
 'TUDCA3Sabc',
 'DRBK',
 'HEX7',
 'PI5P4K',
 'FACOAL205x',
 'DAGKn',
 'NMNATr',
 'PROSTGE2t3',
 'FACOAL2

In [6]:
pruned_model

Name,iCHO3595
Memory address,15cf7a960
Number of metabolites,4764
Number of reactions,7799
Number of genes,2963
Number of groups,0
Objective expression,1.0*biomass_cho_s - 1.0*biomass_cho_s_reverse_dd74e
Compartments,"m, x, c, r, e, n, l, im, g"


### Test

In [8]:
with pruned_model as model:
    objective = 'biomass_cho_s'
    model.objective = objective
    '''
    model.reactions.SCP22x.knock_out()
    model.reactions.TMNDNCCOAtx.knock_out()
    model.reactions.OCCOAtx.knock_out()
    model.reactions.TMNDNCCOAtx.knock_out()
    model.reactions.r0391.knock_out()
    model.reactions.BiGGRxn67.knock_out()
    model.reactions.r2247.knock_out()
    model.reactions.r2280.knock_out()
    model.reactions.r2246.knock_out()

    
    model.reactions.r2279.knock_out()
    model.reactions.r2245.knock_out()
    model.reactions.r2305.knock_out()
    model.reactions.r2317.knock_out()
    model.reactions.r2335.knock_out()
    model.reactions.HMR_0293.knock_out()
    model.reactions.HMR_7741.knock_out()
    model.reactions.r0509.knock_out()
    model.reactions.r1453.knock_out()
    model.reactions.HMR_4343.knock_out()

    
    model.reactions.ACONTm.knock_out()
    model.reactions.PDHm.knock_out()
    model.reactions.r0426.knock_out()
    model.reactions.r0383.knock_out()
    model.reactions.r0555.knock_out()
    model.reactions.r1393.knock_out()
    model.reactions.NICRNS.knock_out()
    model.reactions.get_by_id('GAUGE-R00648').knock_out()
    model.reactions.get_by_id('GAUGE-R03326').knock_out()
    model.reactions.get_by_id('GapFill-R08726').knock_out()

    
    model.reactions.RE2915M.knock_out()
    model.reactions.HMR_3288.knock_out()
    model.reactions.HMR_1325.knock_out()
    model.reactions.HMR_7599.knock_out()
    model.reactions.r1431.knock_out()
    model.reactions.r1433.knock_out()
    model.reactions.RE2439C.knock_out()
    model.reactions.r0082.knock_out()
    model.reactions.r0791.knock_out()
    model.reactions.r1450.knock_out()

    
    model.reactions.get_by_id('GAUGE-R00270').knock_out()
    model.reactions.get_by_id('GAUGE-R02285').knock_out()
    model.reactions.get_by_id('GAUGE-R04283').knock_out()
    model.reactions.get_by_id('GAUGE-R06127').knock_out()
    model.reactions.get_by_id('GAUGE-R06128').knock_out()
    model.reactions.get_by_id('GAUGE-R06238').knock_out()
    model.reactions.get_by_id('GAUGE-R00524').knock_out()
    model.reactions.RE3477C.knock_out()

    model.reactions.AAPSAS.knock_out()
    model.reactions.RE3347C.knock_out()
    model.reactions.HMR_0960.knock_out()
    model.reactions.HMR_0980.knock_out()
    model.reactions.RE3476C.knock_out()
    model.reactions.r0708.knock_out()
    model.reactions.r0777.knock_out()
    model.reactions.r0084.knock_out()
    model.reactions.r0424.knock_out()
    model.reactions.r0698.knock_out()

    model.reactions.get_by_id('3HDH260p').knock_out()
    model.reactions.HMR_3272.knock_out()
    model.reactions.ACOAD183n3m.knock_out()
    model.reactions.HMR_1996.knock_out()
    model.reactions.get_by_id('GapFill-R01463').knock_out()
    model.reactions.get_by_id('GapFill-R04807').knock_out()
    model.reactions.r1468.knock_out()
    model.reactions.r2435.knock_out()
    model.reactions.r0655.knock_out()
    model.reactions.r0603.knock_out()

    model.reactions.r0541.knock_out()
    model.reactions.RE0383C.knock_out()
    model.reactions.HMR_1329.knock_out()
    model.reactions.TYRA.knock_out()
    model.reactions.NRPPHRt_2H.knock_out()
    model.reactions.get_by_id('GAUGE-R07364').knock_out()
    model.reactions.get_by_id('GapFill-R03599').knock_out()
    model.reactions.ARD.knock_out()
    model.reactions.RE3095C.knock_out()
    model.reactions.RE3104C.knock_out()
    model.reactions.RE3104R.knock_out()

    model.reactions.ACONT.knock_out()
    model.reactions.ACONTm.knock_out()
    model.reactions.ICDHxm.knock_out()
    model.reactions.ICDHy.knock_out()
    model.reactions.AKGDm.knock_out()
    model.reactions.r0083.knock_out()
    model.reactions.r0425.knock_out()
    model.reactions.r0556.knock_out()
    model.reactions.NH4t4r.knock_out()
    model.reactions.PROPAT4te.knock_out()
    
    model.reactions.r0085.knock_out()
    model.reactions.r0156.knock_out()
    model.reactions.r0464.knock_out()
    model.reactions.ABUTDm.knock_out()
    model.reactions.OIVD1m.knock_out()
    model.reactions.OIVD2m.knock_out()
    model.reactions.OIVD3m.knock_out()
    model.reactions.r2194.knock_out()
    model.reactions.r2202.knock_out()
    model.reactions.HMR_9617.knock_out()

    model.reactions.r2197.knock_out()
    model.reactions.r2195.knock_out()
    model.reactions.get_by_id('2OXOADOXm').knock_out()
    model.reactions.r2328.knock_out()
    model.reactions.r0386.knock_out()
    model.reactions.r0451.knock_out()
    model.reactions.FAS100COA.knock_out()
    model.reactions.FAS120COA.knock_out()
    model.reactions.FAS140COA.knock_out()
    model.reactions.FAS80COA_L.knock_out()

    model.reactions.r0604.knock_out()
    model.reactions.r0670.knock_out()
    model.reactions.r2334.knock_out()
    model.reactions.r0193.knock_out()
    model.reactions.r0595.knock_out()
    model.reactions.r0795.knock_out()
    model.reactions.GLYCLm.knock_out()
    model.reactions.MACACI.knock_out()
    model.reactions.r2193.knock_out()
    model.reactions.r0779.knock_out()
    model.reactions.r0669.knock_out()
    model.reactions.UDCHOLt.knock_out()
    model.reactions.r2146.knock_out()
    model.reactions.r2139.knock_out()
    model.reactions.r0391.knock_out()
    '''
    for rxn in model.boundary:
        if rxn.id in ['EX_h2o_e', 'EX_h_e', 'EX_o2_e', 'EX_hco3_e', 'EX_so4_e', 'EX_pi_e']:
            rxn.bounds = (-1000, 1000)
        elif rxn.id in ['SK_Asn_X_Ser_Thr_r', 'SK_Tyr_ggn_c', 'SK_Ser_Thr_g', 'SK_pre_prot_r']:
            rxn.bounds = (-0.001, 1000)
        elif rxn.id.startswith(("EX_", "SK_", "DM_")):
            rxn.bounds = (0, 1000)  # Close uptake rates for others

    for reaction in model.reactions:
        if reaction.id == 'EX_etoh_e': #Model creates infeasible solutions when secreting etoh
            continue
        elif reaction.id == 'ATPM': # Add ATP Maintenance Cost
            reaction.lower_bound = 500
        for r,v in uptsec_zela.items():
            if reaction.id == r:
                reaction.upper_bound = 1000
                reaction.lower_bound = v[('U7','P2 to P4')]
    pfba_solution = cobra.flux_analysis.pfba(model)
    pfba_obj = pfba_solution.fluxes[objective]
    pfba_atp = pfba_solution.fluxes['ATPM']
    print(f'Simulated ATP Maintencance is: {pfba_atp}')
    print(f'Simulated growth rate is: {pfba_obj}')
    
    for i,f in pfba_solution.fluxes.items():
        if abs(f) > 1:
            print(i,f)

Simulated ATP Maintencance is: 500.0
Simulated growth rate is: 0.050002710053965664
ATPM 500.0
GLCt4 -50.0098406033136
H2Ot 49.58103028261524
PIt7 16.303811841220877
r1423 16.2836383548553
GGNG 43.85160260135318
GLBRAN 50.11795207241856
GLDBRAN 50.116667702808115
GLGNS1 50.11795207241856
GLPASE1 50.116667702808115
GLPASE2a 43.85060260135318
r1393 6.2660651014549344
NDPK1 -500.8293586784184
UMPK4 -501.43655853758617
IMAGH -25.008646870719126
BiGGRxn67 -501.54640451851327
UGLT -501.16667702808115
EX_h2o_e -24.58181919747657
EX_isomal_e 25.008646870719126


## 3. Biomass prediction using exp. data <a id='biomass'></a>

In [ ]:
import os
import math
import pickle
import pandas as pd

import cobra
from cobra import Reaction
from cobra.io import load_json_model, read_sbml_model, load_matlab_model
from cobra.exceptions import Infeasible
from cobra.sampling import ACHRSampler

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

### Model Loading

In [ ]:
# Directory containing context-specific models models
model_directory = '../Data/Context_specific_models/'

# Model Names
wt_model_names = ['WT_P4_Bio141', 'WT_P4_Bio142', 'WT_P4_Bio143',
                  'WT_P6_Bio141', 'WT_P6_Bio142', 'WT_P6_Bio143']

zela_model_names = ['ZeLa_P2_Bio145', 'ZeLa_P2_Bio146', 'ZeLa_P2_Bio147', 'ZeLa_P2_Bio148',
                    'ZeLa_P4_Bio144', 'ZeLa_P4_Bio145', 'ZeLa_P4_Bio146', 'ZeLa_P4_Bio147', 'ZeLa_P4_Bio148',
                    'ZeLa_P6_Bio144', 'ZeLa_P6_Bio146', 'ZeLa_P6_Bio147', 'ZeLa_P6_Bio148',
                    'ZeLa_P8_Bio144', 'ZeLa_P8_Bio145', 'ZeLa_P8_Bio146', 'ZeLa_P8_Bio147', 'ZeLa_P8_Bio148']

# Dictionaries to store the loaded models
wt_models = {}
zela_models = {}

# List all .mat files in the directory
model_files = [f for f in os.listdir(model_directory) if f.endswith('.mat')]

# Iterate through the .mat files and load only the models that match predefined strings
for model_file in model_files:
    # Full path to the model file
    model_path = os.path.join(model_directory, model_file)
    
    # Check if the file corresponds to a WT model
    for model_name in wt_model_names:
        if model_name in model_file:
            wt_models[model_name] = load_matlab_model(model_path)
            wt_models[model_name].id = model_name
            print(f"Loaded WT model: {model_name}")
            break  # Stop checking other names since this file is already processed
    
    # Check if the file corresponds to a ZeLa model
    for model_name in zela_model_names:
        if model_name in model_file:
            zela_models[model_name] = load_matlab_model(model_path)
            zela_models[model_name].id = model_name
            print(f"Loaded ZeLa model: {model_name}")
            break  # Stop checking other names since this file is already processed

In [ ]:
# ATP loop generating reactions

atp_reactions_list = [
    'SCP22x', 'TMNDNCCOAtx', 'OCCOAtx', 'r0391', 'BiGGRxn67', 'r2247', 'r2280', 'r2246', 'r2279', 'r2245', 
    'r2305', 'r2317', 'r2335', 'HMR_0293', 'HMR_7741', 'r0509', 'r1453', 'HMR_4343', 'ACONTm', 'PDHm', 
    'r0426', 'r0383', 'r0555', 'r1393', 'NICRNS', 'GAUGE-R00648', 'GapFill-R08726', 
    'RE2915M', 'HMR_3288', 'HMR_1325', 'HMR_7599', 'r1431', 'r1433', 'RE2439C', 'r0082', 'r0791', 'r1450', 
    'GAUGE-R00270', 'GAUGE-R02285', 'GAUGE-R04283', 'GAUGE-R06127', 'GAUGE-R06128', 'GAUGE-R06238', 
    'GAUGE-R00524', 'GAUGE-R03326', 'RE3477C', 'AAPSAS', 'RE3347C', 'HMR_0960', 'HMR_0980', 'RE3476C', 
    'r0708', 'r0777', 'r0084', 'r0424', 'r0698', '3HDH260p', 'HMR_3272', 'ACOAD183n3m', 'HMR_1996', 
    'GapFill-R01463', 'GapFill-R04807', 'r1468', 'r2435', 'r0655', 'r0603', 'r0541', 'RE0383C', 'HMR_1329', 
    'TYRA', 'NRPPHRt_2H', 'GAUGE-R07364', 'GapFill-R03599', 'ARD', 'RE3095C', 'RE3104C', 'RE3104R', 'ACONT', 
    'ACONTm', 'ICDHxm', 'ICDHy', 'AKGDm', 'r0083', 'r0425', 'r0556', 'NH4t4r', 'PROPAT4te', 'r0085', 
    'r0156', 'r0464', 'ABUTDm', 'OIVD1m', 'OIVD2m', 'OIVD3m', 'r2194', 'r2202', 'HMR_9617', 'r2197', 
    'r2195', '2OXOADOXm', 'r2328', 'r0386', 'r0451', 'FAS100COA', 'FAS120COA', 'FAS140COA', 'FAS80COA_L', 
    'r0604', 'r0670', 'r2334', 'r0193', 'r0595', 'r0795', 'GLYCLm', 'MACACI', 'r2193', 'r0779', 'r0669', 
    'UDCHOLt', 'r2146', 'r2139'
]

### Simulations

In [ ]:
# Load uptake and secretion rate dict

with open('../Data/Uptake_Secretion_Rates/uptake_secretion_raw_wt_dict.pkl', 'rb') as file:
    uptsec_wt = pickle.load(file)

with open('../Data/Uptake_Secretion_Rates/uptake_secretion_raw_zela_dict.pkl', 'rb') as file:
    uptsec_zela = pickle.load(file)

In [ ]:
# Sanity Check. Make sure all the exchange reactions in the experimental data are present in the model and that the models ar growing

for name,model in zela_models.items():
    print('---------------------------------------')
    print(f'------------{model.id}------------')
    print('')
    print(f'Objective:{model.objective}')
    print('')
    print(f'GR1: {model.slim_optimize()}')
    exchanges = list(uptsec_wt.keys())
    
    model_ex = [r.id for r in model.boundary]
    model_reactions = [r.id for r in model.reactions]
    
    # Remove ATP loop reactions
    for atp_reaction in atp_reactions_list:
        if atp_reaction in model_reactions:
            # Knock out the reaction temporarily
            reaction = model.reactions.get_by_id(atp_reaction)
            original_bounds = reaction.bounds  # Store the original bounds to revert later
            reaction.knock_out()
            
            # Test if the model remains feasible after knocking out the reaction
            try:
                new_gr = model.slim_optimize()
                print(f'GR2 (After knocking out {atp_reaction}): {new_gr}')
                
                # Check if the model's growth rate is infeasible or close to zero
                if math.isnan(new_gr) or new_gr < 1e-6:
                    print(f'{atp_reaction} causes infeasibility. Reverting knockout.')
                    reaction.bounds = original_bounds  # Revert the bounds if infeasible
                else:
                    print(f'{atp_reaction} successfully knocked out without infeasibility.')

            except Exception as e:
                print(f'Error optimizing model after knocking out {atp_reaction}: {e}')
                # Revert the reaction bounds if an error occurs
                reaction.bounds = original_bounds

    print('')
    print('Missing reactions from exp data:')
    for ex in exchanges:
        if ex not in model_ex:
            print(ex)
    print('---------------------------------------')
    print('---------------------------------------')
    print('')
    print('')


for name,model in wt_models.items():
    print('---------------------------------------')
    print(f'------------{model.id}------------')
    print('')
    print(f'Objective:{model.objective}')
    print('---------------------------------------')
    print(f'GR: {model.slim_optimize()}')
    exchanges = list(uptsec_wt.keys())
    model_ex = [r.id for r in model.boundary]
    model_reactions = [r.id for r in model.reactions]
    
    # Remove ATP loop reactions
    for atp_reaction in atp_reactions_list:
        if atp_reaction in model_reactions:
            # Knock out the reaction temporarily
            reaction = model.reactions.get_by_id(atp_reaction)
            original_bounds = reaction.bounds  # Store the original bounds to revert later
            reaction.knock_out()
            
            # Test if the model remains feasible after knocking out the reaction
            try:
                new_gr = model.slim_optimize()
                print(f'GR2 (After knocking out {atp_reaction}): {new_gr}')
                
                # Check if the model's growth rate is infeasible or close to zero
                if math.isnan(new_gr) or new_gr < 1e-6:
                    print(f'{atp_reaction} causes infeasibility. Reverting knockout.')
                    reaction.bounds = original_bounds  # Revert the bounds if infeasible
                else:
                    print(f'{atp_reaction} successfully knocked out without infeasibility.')

            except Exception as e:
                print(f'Error optimizing model after knocking out {atp_reaction}: {e}')
                # Revert the reaction bounds if an error occurs
                reaction.bounds = original_bounds

    
    for ex in exchanges:
        if ex not in model_ex:
            print(ex)
    print('---------------------------------------')
    print('---------------------------------------')
    print('')

In [ ]:
pfba_solutions_fluxes = []

In [ ]:
# WT

results = []

objective = 'biomass_cho_s'

# Adjust the lower bound values according to the experimental growth rates in order to indetify bottlenecks
intervals = {'P0 to P2':'P2', 'P2 to P4':'P4', 'P4 to P6':'P6', 'P6 to P8':'P8', 'P8 to P12':'P12', 'P12 to P14':'P14'}
replicates = {'U1':'Bio141', 'U2':'Bio142', 'U3':'Bio143'}

for name,model in wt_models.items():
    print(f"Processing model: {name}")
    # Set lower bounds of the reactions according to the experimental data
    for interval_key, interval_model_time in intervals.items():
        if interval_model_time in name:  # Match model name with interval
            for rep_key, rep_suffix in replicates.items():
                if rep_suffix in name:  # Match replicate with model name

                     with model as modified_model:

                        # Open the bounds for the biomass reaction 
                        modified_model.reactions.biomass_cho_s.bounds = (0,1000)
                         
                        # Create a copy of the modified_model before making changes
                        pre_modification_model = modified_model.copy()
                         
                        for rxn in modified_model.boundary:
                            # Keep boundaries open for essential metabolites
                            if rxn.id in ['EX_h2o_e', 'EX_h_e', 'EX_o2_e', 'EX_hco3_e', 'EX_so4_e', 'EX_pi_e']:
                                rxn.bounds = (-1000, 1000)
                            elif rxn.id in ['SK_Asn_X_Ser_Thr_r', 'SK_Tyr_ggn_c', 'SK_Ser_Thr_g', 'SK_pre_prot_r']:
                                rxn.bounds = (-0.001, 1000)
                            elif rxn.id.startswith(("EX_", "SK_", "DM_")):
                                rxn.bounds = (0, 1000)  # Close uptake rates for others
                        
                        print(f'Calculating Growth Rate for WT Condition:{rep_key,interval_key}')
                        exp_gr = uptsec_wt['exp_growth_rate'][(rep_key,interval_key)]
                        print(f'Experimental growth rate is: {exp_gr}')
                        for reaction in modified_model.reactions:
                            if reaction.id == 'EX_etoh_e': #Model creates infeasible solutions when secreting etoh
                                continue
                            elif reaction.id == 'ATPM': # Add ATP Maintenance Cost
                                reaction.lower_bound = 8
                            for r,v in uptsec_wt.items():
                                if reaction.id == r:
                                    reaction.upper_bound = 1000
                                    reaction.lower_bound = v[(rep_key,interval_key)]
                                    
                        try:
                            pfba_solution = cobra.flux_analysis.pfba(modified_model)
                            pfba_obj = pfba_solution.fluxes[objective]
                            pfba_atp = pfba_solution.fluxes['ATPM']
                            print(f'Simulated ATP Maintencance is: {pfba_atp}')
                            print(f'Simulated growth rate is: {pfba_obj}')
            
                            # If pFBA succeeds, proceed to sampling
                            pfba_model_to_sample = modified_model
                            
                        except Infeasible:
                            print(f'Infeasible solution for replicate {rep_key} interval {interval_key}')
                            print('Reverting to the original model and retrying pFBA...')

                            for rxn in pre_modification_model.boundary:
                            # Keep boundaries open for essential metabolites
                                if rxn.id in ['EX_h2o_e', 'EX_h_e', 'EX_o2_e', 'EX_hco3_e', 'EX_so4_e', 'EX_pi_e']:
                                    rxn.bounds = (-1000, 1000)
                                elif rxn.id in ['SK_Asn_X_Ser_Thr_r', 'SK_Tyr_ggn_c', 'SK_Ser_Thr_g', 'SK_pre_prot_r']:
                                    rxn.bounds = (-0.001, 1000)
                            
                            pfba_solution = cobra.flux_analysis.pfba(pre_modification_model)
                            pfba_obj = pfba_solution.fluxes[objective]
                            pfba_atp = pfba_solution.fluxes['ATPM']
                            print(f'Simulated ATP Maintencance is: {pfba_atp}')
                            print(f'Simulated growth rate with original model is: {pfba_obj}')

                            # If pFBA with original model succeeds, proceed to sampling
                            pfba_model_to_sample = pre_modification_model

                        results.append([rep_suffix, interval_key, exp_gr, pfba_obj])
                        # Save the solution with metadata
                        pfba_solutions_fluxes.append({
                            "model": pre_modification_model.id,
                            "condition": interval_key,
                            "fluxes": pfba_solution.fluxes
                        })

                        
                         # Perform flux sampling using ACHRSampler
                        #print("Performing flux sampling...")
                        #achr_sampler = ACHRSampler(pfba_model_to_sample)
                        #samples = 5000 
                        #sampled_fluxes = achr_sampler.sample(samples)
                        
                        # Validate and save the sampled fluxes
                        #if 'v' not in achr_sampler.validate(sampled_fluxes).flatten():
                        #    print(f"Error in sampling for {rep_key}, {interval_key}")
                        #else:
                        #    f_name = f"../Simulations/flux_sampling/Sampling_{name}_{samples}.pkl"
                        #    with open(f_name, 'wb') as file:
                        #        pickle.dump(sampled_fluxes, file)
                        
# Creating a DataFrame
df_wt = pd.DataFrame(results, columns=['Hue', 'Category', 'X Axis', 'Y Axis'])

In [ ]:
df =df_wt
# Define markers for categories
markers = {
    'P0 to P2': 'p',  # Circle
    'P2 to P4': 'o',  # Circle
    'P4 to P6': 's',   # Square
}

# Set up the plot
plt.figure(figsize=(10, 6))

# Plot each group with different colors for hue and different shapes for categories
hues = df['Hue'].unique()
colors = plt.colormaps.get_cmap('tab10')

for hue_idx, hue in enumerate(hues):
    hue_subset = df[df['Hue'] == hue]
    for category, marker in markers.items():
        subset = hue_subset[hue_subset['Category'] == category]
        plt.scatter(subset['X Axis'], subset['Y Axis'], label=f'{hue} - {category}', marker=marker, s=100, color=colors(hue_idx))

# Add the identity line y = x
plt.plot([0.00, 0.055], [0.00, 0.055], color='red', linestyle='--')

# Set evenly distributed ticks from 0.01 to 0.08
ticks = [i / 100.0 for i in range(1, 11)]
plt.xticks(ticks=ticks, labels=[f'{i/100.0:.2f}' for i in range(1, 11)])
plt.yticks(ticks=ticks, labels=[f'{i/100.0:.2f}' for i in range(1, 11)])

# Set limits for both axes
plt.xlim(0.0, 0.055)
plt.ylim(0.0, 0.055)

# Setting the same scale for both axes
plt.gca().set_aspect('equal', adjustable='box')

# Adding labels and title
plt.xlabel('Experimental Growth Rate')
plt.ylabel('Simulated Growth Rate')
plt.title('WT')
plt.legend(title='Hue - Category', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.tight_layout()
plt.savefig('../Simulations/growth_rate_pred/bar_plot_growth_rate_wt.png')
plt.show()

In [ ]:
# ZeLa

results = []

objective = 'biomass_cho_s'

# Adjust the lower bound values according to the experimental growth rates in order to indetify bottlenecks
intervals = {'P0 to P2':'P2', 'P2 to P4':'P4', 'P4 to P6':'P6', 'P6 to P8':'P8', 'P8 to P12':'P12', 'P12 to P14':'P14'}
replicates = {'U4':'Bio144', 'U5':'Bio145', 'U6':'Bio146', 'U7':'Bio147', 'U8':'Bio148'}

for name,model in zela_models.items():
    print(f"Processing model: {name}")
    # Set lower bounds of the reactions according to the experimental data
    for interval_key, interval_model_time in intervals.items():
        if interval_model_time in name:  # Match model name with interval
            for rep_key, rep_suffix in replicates.items():
                if rep_suffix in name:  # Match replicate with model name
                    
                     with model as modified_model:

                        # Open the bounds for the biomass reaction 
                        modified_model.reactions.biomass_cho_s.bounds = (0,1000)
                         
                        # Create a copy of the modified_model before making changes
                        pre_modification_model = modified_model.copy()
 
                        for rxn in modified_model.boundary:
                            if rxn.id in ['EX_h2o_e', 'EX_h_e', 'EX_o2_e', 'EX_hco3_e', 'EX_so4_e', 'EX_pi_e']:
                                rxn.bounds = (-1000, 1000)
                            elif rxn.id in ['SK_Asn_X_Ser_Thr_r', 'SK_Tyr_ggn_c', 'SK_Ser_Thr_g', 'SK_pre_prot_r']:
                                rxn.bounds = (-0.001, 1000)
                            elif rxn.id.startswith(("EX_", "SK_", "DM_")):
                                rxn.bounds = (0, 1000)  # Close uptake rates for others

                        print(f'Calculating Growth Rate for ZeLa Condition:{rep_key,interval_key}')
                        exp_gr = uptsec_zela['exp_growth_rate'][(rep_key,interval_key)]
                        print(f'Experimental growth rate is: {exp_gr}')
                        for reaction in modified_model.reactions:
                            if reaction.id == 'EX_etoh_e': #Model creates infeasible solutions when secreting etoh
                                continue
                            elif reaction.id == 'ATPM': # Add ATP Maintenance Cost
                                reaction.lower_bound = 8
                            for r,v in uptsec_zela.items():
                                if reaction.id == r:
                                    reaction.upper_bound = 1000
                                    reaction.lower_bound = v[(rep_key,interval_key)]
                                    
                        try:
                            pfba_solution = cobra.flux_analysis.pfba(modified_model)
                            pfba_obj = pfba_solution.fluxes[objective]
                            pfba_atp = pfba_solution.fluxes['ATPM']
                            print(f'Simulated ATP Maintencance is: {pfba_atp}')
                            print(f'Simulated growth rate is: {pfba_obj}')
                            
                        except Infeasible:
                            print(f'Infeasible solution for replicate {rep_key} interval {interval_key}')
                            print('Reverting to the original model and retrying pFBA...')

                            for rxn in pre_modification_model.reactions:
                                if rxn.id in ['EX_h2o_e', 'EX_h_e', 'EX_o2_e', 'EX_hco3_e', 'EX_so4_e', 'EX_pi_e']:
                                    rxn.bounds = (-1000, 1000)
                                elif rxn.id in ['SK_Asn_X_Ser_Thr_r', 'SK_Tyr_ggn_c', 'SK_Ser_Thr_g', 'SK_pre_prot_r']:
                                    rxn.bounds = (-0.001, 1000)
                                elif rxn.id == 'ATPM': # Add ATP Maintenance Cost
                                    rxn.lower_bound = 8
                                
                            pfba_solution = cobra.flux_analysis.pfba(pre_modification_model)
                            pfba_obj = pfba_solution.fluxes[objective]
                            pfba_atp = pfba_solution.fluxes['ATPM']
                            print(f'Simulated ATP Maintencance is: {pfba_atp}')
                            print(f'Simulated growth rate with original model is: {pfba_obj}')
                            
                        results.append([rep_suffix, interval_key, exp_gr, pfba_obj])

                        # Save the solution with metadata
                        pfba_solutions_fluxes.append({
                            "model": pre_modification_model.id,
                            "condition": interval_key,
                            "fluxes": pfba_solution.fluxes
                        })
                         
                        '''
                        # Perform flux sampling using ACHRSampler
                        print("Performing flux sampling...")
                        achr_sampler = ACHRSampler(pfba_model_to_sample)
                        samples = 1 
                        sampled_fluxes = achr_sampler.sample(samples)
                        
                        # Validate and save the sampled fluxes
                        if 'v' not in achr_sampler.validate(sampled_fluxes).flatten():
                            print(f"Error in sampling for {rep_key}, {interval_key}")
                        else:
                            f_name = f"../Simulations/flux_sampling/Sampling_{name}_{samples}.pkl"
                            with open(f_name, 'wb') as file:
                                pickle.dump(sampled_fluxes, file)
                        '''
    
# Creating a DataFrame
df_zela = pd.DataFrame(results, columns=['Hue', 'Category', 'X Axis', 'Y Axis'])

In [ ]:
df =df_zela
# Define markers for categories
markers = {
    'P0 to P2': 'p',
    'P2 to P4': 'o',  # Circle
    'P4 to P6': 's',   # Square
    'P6 to P8': 'v',
}

# Set up the plot
plt.figure(figsize=(10, 6))

# Plot each group with different colors for hue and different shapes for categories
hues = df['Hue'].unique()
colors = plt.colormaps.get_cmap('tab10')

for hue_idx, hue in enumerate(hues):
    hue_subset = df[df['Hue'] == hue]
    for category, marker in markers.items():
        subset = hue_subset[hue_subset['Category'] == category]
        plt.scatter(subset['X Axis'], subset['Y Axis'], label=f'{hue} - {category}', marker=marker, s=100, color=colors(hue_idx))

# Add the identity line y = x
plt.plot([0.00, 0.055], [0.00, 0.055], color='red', linestyle='--')

# Set evenly distributed ticks from 0.01 to 0.08
ticks = [i / 100.0 for i in range(1,11)]
plt.xticks(ticks=ticks, labels=[f'{i/100.0:.2f}' for i in range(1, 11)])
plt.yticks(ticks=ticks, labels=[f'{i/100.0:.2f}' for i in range(1, 11)])

# Set limits for both axes
plt.xlim(0.0, 0.055)
plt.ylim(0.0, 0.055)

# Setting the same scale for both axes
plt.gca().set_aspect('equal', adjustable='box')

# Adding labels and title
plt.xlabel('Experimental Growth Rate')
plt.ylabel('Simulated Growth Rate')
plt.title('ZeLa')
plt.legend(title='Hue - Category', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.tight_layout()
plt.savefig('../Simulations/growth_rate_pred/bar_plot_growth_rate_zela.png')
plt.show()

In [ ]:
for reaction, values in uptsec_zela.items():
    plt.figure(figsize=(10, 6))
    
    # Organize data for plotting
    batches = sorted(set(k[0] for k in values.keys()))
    timepoints = ['P0 to P2', 'P2 to P4', 'P4 to P6', 'P6 to P8', 'P8 to P12', 'P12 to P14']
    
    for batch in batches:
        batch_data = [values.get((batch, tp), 0) for tp in timepoints]
        plt.plot(timepoints, batch_data, marker='o', label=batch)
    
    plt.title(f'Changes in {reaction} over time')
    plt.xlabel('Timepoints')
    plt.ylabel('Values')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

In [ ]:
for reaction, values in uptsec_wt.items():
    plt.figure(figsize=(10, 6))
    
    # Organize data for plotting
    batches = sorted(set(k[0] for k in values.keys()))
    timepoints = ['P0 to P2', 'P2 to P4', 'P4 to P6', 'P6 to P8', 'P8 to P12', 'P12 to P14']
    
    for batch in batches:
        batch_data = [values.get((batch, tp), 0) for tp in timepoints]
        plt.plot(timepoints, batch_data, marker='o', label=batch)
    
    plt.title(f'Changes in {reaction} over time')
    plt.xlabel('Timepoints')
    plt.ylabel('Values')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

## 4. Flux Enrichment Analysis <a id='fea'></a>

In [ ]:
# OPTION 1
# Use the fluxes from the growth rate calculations to filter reactions with fluxes
# Define a cut off for the flux ??? # Plot to visualization of the flux distribution
# Run Flux Enrichment Analysis on those reactions to see pathways enriched when optimized by biomass

In [ ]:
# OPTION 2
# Transcriptomic data from cell batches / biorreactiors
# Overlay this into the recons and extract the reactions associated to the genes
# Extract a reaction vector / 
# Run Flux Enrichment Analysis

In [ ]:
import cobra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests

def flux_enrichment_analysis(generic_model, rxn_list, attribute='subsystem'):
    
    # Check if the attribute exists in the first reaction as a proxy for all
    if not hasattr(model.reactions[0], attribute):
        raise ValueError(f'Attribute {attribute} not found in model reactions')

    # Extract attribute information for all reactions
    attribute_values = [getattr(rxn, attribute, 'None') for rxn in model.reactions]
    unique_attributes = set(attribute_values)
    
    # Count occurrences in the model and in the reaction set
    model_counts = {attr: attribute_values.count(attr) for attr in unique_attributes}
    rxn_set_counts = {attr: 0 for attr in unique_attributes}
    for rxn in rxn_list:
        rxn_attr = getattr(model.reactions.get_by_id(rxn), attribute, 'None')
        rxn_set_counts[rxn_attr] += 1

    # Calculate p-values using hypergeometric test
    M = len(model.reactions)  # Total number of reactions
    n = len(rxn_list)  # Size of reaction set
    p_values = []
    for attr in unique_attributes:
        N = model_counts[attr]  # Total reactions in group
        x = rxn_set_counts[attr]  # Reactions in group and in set
        p_value = hypergeom.sf(x-1, M, N, n)
        p_values.append(p_value)

    # Adjust p-values for multiple testing
    _, adj_p_values, _, _ = multipletests(p_values, method='fdr_bh')

    # Compile results
    results = pd.DataFrame({
        'Group': list(unique_attributes),
        'P-value': p_values,
        'Adjusted P-value': adj_p_values,
        'Enriched set size': [rxn_set_counts[attr] for attr in unique_attributes],
        'Total set size': [model_counts[attr] for attr in unique_attributes],
    }).sort_values(by='Adjusted P-value')

    return results

### Test 1: FEA on reactions active during growth rate optimization
Use the fluxes from the growth rate calculations to filter reactions with fluxes

In [ ]:
# Generate a vector of active reactions for each one of the conditions

active_reactions = []

for sol in pfba_solutions_fluxes:
    rxns_fluxes = []
    for n,f in sol['fluxes'].items():
        if f != 0:
            rxns_fluxes.append(n)
    
    active_reactions.append({
        "batch": sol['model'],
        "condition": sol['condition'],
        "flux_vector": rxns_fluxes
    })


In [ ]:
# Load generic model
model = load_json_model('iCHO3595_unblocked.json')

# Add atp demand reaction
DM_atp = Reaction('DM_atp_c')
model.add_reactions([DM_atp])
model.reactions.DM_atp_c.build_reaction_from_string('atp_c -->')
model.reactions.DM_atp_c.bounds = (0.01, 1000)

In [ ]:
# Generate FEA results for each one of the vectors generated for each condition

fea_results = []

# Load generic model
#model = load_json_model('iCHO3595_unblocked.json')

for fluxes in active_reactions:
    results = flux_enrichment_analysis(model, fluxes['flux_vector'], 'subsystem')
    fea_results.append({
        "Batch": fluxes['batch'],
        "Condition": fluxes['condition'],
        "Results": results
    })

In [ ]:
# Transform p-values to -log10

all_data = pd.DataFrame()

for result in fea_results:
    batch = result['Batch']
    temp_df = result['Results'][['Group', 'P-value']].copy()
    temp_df.columns = ['Group', f'P-value_{batch}']
    if all_data.empty:
        all_data = temp_df
    else:
        all_data = pd.merge(all_data, temp_df, on='Group', how='outer')

# Remove groups with a 0 value in all conditions (assuming a '0' value indicates non-significance)
significant_filter = (all_data.drop(columns='Group') < 0.05).any(axis=1)
filtered_data = all_data[significant_filter]
filtered_data.reset_index(inplace=True, drop=True)

# Transform the p-values
for col in filtered_data.columns:
    if col.startswith('P-value'):
        filtered_data[f'-log10_{col}'] = -np.log10(filtered_data[col])

In [ ]:
from scipy.stats import zscore

# Extract relevant columns for heatmap
heatmap_data = filtered_data[[col for col in filtered_data.columns if col.startswith('-log10')]]

# Clean column names
heatmap_data.columns = heatmap_data.columns.str.replace(r"-log10_P-value_", "", regex=True)

# Normalize the data (z-score)
normalized_data = heatmap_data.apply(zscore, axis=1)

# Prepare heatmap data
normalized_data.index = filtered_data['Group']

# Reorder columns: Group P2, P4, P6, and P8 together
columns_order = []
for phase in ['P2', 'P4', 'P6', 'P8']:
    columns_order.extend([col for col in normalized_data.columns if phase in col])

# Reorder the DataFrame
normalized_data = normalized_data[columns_order]

# Plotting the heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(normalized_data, annot=True, cmap="viridis", cbar_kws={'label': 'Z-score of -log10(P-value)'}, vmin=-1.5, vmax=1.5)
plt.title('Heatmap of Z-score Normalized -log10(P-values) Across Conditions', fontsize=20)
plt.xlabel('Conditions', fontsize=15)
plt.ylabel('Groups', fontsize=15)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

plt.savefig('../Simulations/flux_enrichment_analysis/FEA_heatmap_all.png', format='png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Filter the columns based on the presence of "P2 to P4" or "P4 to P6"
subset_df = heatmap_data.filter(regex='_P4_|_P6_')

# Normalize the data (z-score)
normalized_data = subset_df.apply(zscore, axis=1)

# Prepare heatmap data
normalized_data.index = filtered_data['Group']

# Plotting the heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(normalized_data, annot=True, cmap="viridis", cbar_kws={'label': 'Z-score of -log10(P-value)'}, vmin=-1.5, vmax=1.5)
plt.title('Heatmap of Z-score Normalized -log10(P-values) Across Conditions', fontsize=20)
plt.xlabel('Conditions', fontsize=15)
plt.ylabel('Groups', fontsize=15)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

plt.savefig('../Simulations/flux_enrichment_analysis/FEA_heatmap_p2_p6.png', format='png', dpi=300, bbox_inches='tight')
plt.show()